In [14]:
from bs4 import BeautifulSoup
import requests
from urllib.parse import urljoin
import pandas as pd

In [2]:
! pip install apify_client


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [15]:
from apify_client import ApifyClient

APIFY_TOKEN = 'apify_api_gxHBZNaeyfZ2uXbe5TmycB92jAtYRx0sO7ZQ'  # Replace with your real token
client = ApifyClient(APIFY_TOKEN)

In [16]:
actor_id = "compass/crawler-google-places"

In [17]:
run_input = {
    "includeWebResults": False,
    "language": "en",
    "locationQuery": "Los Angeles",
    "maxCrawledPlacesPerSearch": 50,
    "maxImages": 0,
    "maximumLeadsEnrichmentRecords": 0,
    "scrapeContacts": True,
    "scrapeDirectories": False,
    "scrapeImageAuthors": False,
    "scrapePlaceDetailPage": False,
    "scrapeReviewsPersonalData": True,
    "scrapeTableReservationProvider": False,
    "searchMatching": "all",
    "searchStringsArray": ["restaurant"],
    "skipClosedPlaces": False,
    "website": "withWebsite",
    "includeEnrichments": True
}

In [18]:
# Start the actor
run = client.actor(actor_id).call(run_input=run_input)

# Fetch the output dataset
dataset_items = client.dataset(run["defaultDatasetId"]).list_items().items

# Display a few results
for item in dataset_items[:5]:
    print(f"Name: {item.get('title')}")
    print(f"Website: {item.get('website')}")
    print(f"Emails: {item.get('emails')}")
    print(f"Socials: {item.get('socials')}")
    print(f"Phone: {item.get('phone')}")
    print("="*40)

[apify.crawler-google-places runId:MRw10a3GK5q7bFPKD] -> Status: RUNNING, Message: 
[apify.crawler-google-places runId:MRw10a3GK5q7bFPKD] -> 2025-06-23T18:58:10.425Z ACTOR: Pulling Docker image of build TIVsfdCTM4nrhY1m7 from registry.
[apify.crawler-google-places runId:MRw10a3GK5q7bFPKD] -> 2025-06-23T18:58:10.427Z ACTOR: Creating Docker container.
[apify.crawler-google-places runId:MRw10a3GK5q7bFPKD] -> 2025-06-23T18:58:10.482Z ACTOR: Starting Docker container.
[apify.crawler-google-places runId:MRw10a3GK5q7bFPKD] -> 2025-06-23T18:58:12.487Z INFO  System info {"apifyVersion":"3.3.2","apifyClientVersion":"2.12.0","crawleeVersion":"3.13.1","osType":"Linux","nodeVersion":"v18.20.8"}
[apify.crawler-google-places runId:MRw10a3GK5q7bFPKD] -> 2025-06-23T18:58:13.421Z
[apify.crawler-google-places runId:MRw10a3GK5q7bFPKD] -> 2025-06-23T18:58:13.425Z INFO  *** STARTING TO LOOK FOR (GEO)LOCATION ***
[apify.crawler-google-places runId:MRw10a3GK5q7bFPKD] -> 2025-06-23T18:58:13.430Z
[apify.crawler

Name: Denj Restaurant
Website: https://denj-cafe.com/
Emails: []
Socials: None
Phone: (818) 340-7777
Name: Shirin Restaurant
Website: https://shirinrestaurant.com/
Emails: []
Socials: None
Phone: (818) 887-1010
Name: Kourosh Restaurant
Website: http://www.kouroshrestaurant.com/
Emails: ['info@kouroshrestaurant.com']
Socials: None
Phone: (818) 696-5000
Name: Fortune House Restaurant
Website: https://www.fortunehousethairestaurant.com/
Emails: []
Socials: None
Phone: (818) 781-4699
Name: Anarbagh - Woodland Hills Indian Restaurant
Website: https://anarbaghrestaurant.com/
Emails: ['Anarbagh1995@gmail.com']
Socials: None
Phone: (818) 224-3929


In [19]:
import pandas as pd
df = pd.DataFrame(dataset_items)
df.columns

Index(['title', 'description', 'price', 'categoryName', 'address',
       'neighborhood', 'street', 'city', 'postalCode', 'state', 'countryCode',
       'website', 'phone', 'phoneUnformatted', 'claimThisBusiness', 'location',
       'totalScore', 'permanentlyClosed', 'temporarilyClosed', 'placeId',
       'categories', 'fid', 'cid', 'reviewsCount', 'reviewsDistribution',
       'imagesCount', 'imageCategories', 'scrapedAt', 'googleFoodUrl',
       'hotelAds', 'openingHours', 'peopleAlsoSearch', 'placesTags',
       'reviewsTags', 'additionalInfo', 'gasPrices', 'url', 'searchPageUrl',
       'searchString', 'language', 'rank', 'isAdvertisement', 'imageUrl',
       'kgmid', 'domain', 'emails', 'phones', 'phonesUncertain', 'linkedIns',
       'twitters', 'instagrams', 'facebooks', 'youtubes', 'tiktoks',
       'pinterests', 'discords', 'reviews', 'leadsEnrichment',
       'additionalOpeningHours', 'reserveTableUrl'],
      dtype='object')

In [20]:

columns_to_keep = [
    'title', 'address', 'website', 'emails', 'phone', 'location', 'categories','instagrams', 'facebooks']
df_filtered = df[columns_to_keep]

In [26]:
# ----- CAREER PAGE DETECTION UTILITIES -----
third_party_job_sites = [
    "indeed.com", "linkedin.com", "ziprecruiter.com", "glassdoor.com", "greenhouse.io",
    "monster.com", "workday.com", "bamboohr.com", "lever.co"
]

def is_third_party_job_site(url):
    return any(site in url.lower() for site in third_party_job_sites)

def detect_careers_page_from_homepage(website_url):
    try:
        if pd.isna(website_url) or not website_url.startswith("http"):
            return None

        headers = {"User-Agent": "Mozilla/5.0"}
        res = requests.get(website_url, headers=headers, timeout=10)
        soup = BeautifulSoup(res.text, "html.parser")
        text = soup.get_text(" ", strip=True)

        for a in soup.find_all("a", href=True):
            href = a["href"]
            link_text = a.get_text().lower()
            if is_third_party_job_site(href):
                continue
            if any(kw in href.lower() or kw in link_text for kw in [
                "career", "careers", "job", "jobs", "employment", "hiring", "apply", "work with us"
            ]):
                return urljoin(website_url, href)
    except:
        return None
    return None


In [27]:
# Add Career Page column
df_filtered["Career Page"] = df_filtered["website"].apply(detect_careers_page_from_homepage)


/tmp/ipykernel_11134/449532543.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered["Career Page"] = df_filtered["website"].apply(detect_careers_page_from_homepage)


In [28]:
# Safe function to clean URLs
def clean_url_query(url):
    if isinstance(url, list):
        url = "; ".join(url)
    if pd.isna(url) or url == "":
        return url
    return "; ".join(u.split("?")[0] for u in str(url).split(";"))

# Apply safely with .loc
columns_to_clean = ["website", "instagrams", "facebooks","Career Page"]

for col in columns_to_clean:
    if col in df_filtered.columns:
        df_filtered.loc[:, col] = df_filtered[col].apply(clean_url_query)


In [29]:
df_filtered.head()

,title,address,website,emails,phone,location,categories,instagrams,facebooks,Career Page
0,Denj Restaurant,"20002 Ventura Blvd, Woodland Hills, CA 91364",https://denj-cafe.com/,[],(818) 340-7777,"{'lat': 34.1716251, 'lng': -118.5687262}","[Persian restaurant, Takeout Restaurant, Sandw...",,,None
1,Shirin Restaurant,"21826 Ventura Blvd, Woodland Hills, CA 91364",https://shirinrestaurant.com/,[],(818) 887-1010,"{'lat': 34.1681951, 'lng': -118.6030123}","[Persian restaurant, Caterer, Mediterranean re...",https://www.instagram.com/shirinrestaurantla/,https://www.facebook.com/ShirinRestaurant/,None
2,Kourosh Restaurant,"22635 Ventura Blvd Unit A, Woodland Hills, CA ...",http://www.kouroshrestaurant.com/,[info@kouroshrestaurant.com],(818) 696-5000,"{'lat': 34.1677641, 'lng': -118.6204851}","[Persian restaurant, Mediterranean restaurant,...",http://www.instagram.com/farnoshranjbari/,http://www.facebook.com/kourosh,None
3,Fortune House Restaurant,"17206 Saticoy St, Lake Balboa, CA 91406",https://www.fortunehousethairestaurant.com/,[],(818) 781-4699,"{'lat': 34.2078399, 'lng': -118.5080347}","[Thai restaurant, Chinese restaurant]",,,None
4,Anarbagh - Woodland Hills Indian Restaurant,"22721 Ventura Blvd, Woodland Hills, CA 91364",https://anarbaghrestaurant.com/,[Anarbagh1995@gmail.com],(818) 224-3929,"{'lat': 34.166824, 'lng': -118.6215145}","[Restaurant, Indian restaurant]",https://www.instagram.com/anarbaghrestaurant/;...,,None


In [30]:
output_file = "los_angeles_restaurants_scraped.csv"
df_filtered.to_csv(output_file, index=False)